In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras import layers

In [ ]:
model = keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# model.add(layers.Embedding(input_dim=1000, output_dim=64)) # Embedding layer is not needed for this task

model_name = "3wheel_model_lstm_128_tanh_new"
model = keras.Sequential([
    layers.LSTM(128, input_shape=(None, 15), activation='tanh', return_sequences=False),
    layers.Dense(64, activation='relu'),
    layers.Dense(9)  # [Kp, Ki, Kd]
])

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 128)            │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,569 (322.54 KB)

 Trainable params: 82,569 (322.54 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Compile the model
model.compile(optimizer='adam',
              loss='mse',  # Mean Squared Error is a common loss function for regression tasks
              metrics=['mae']) # Mean Absolute Error is another useful metric for regression

def build_sequences(X, y, seq_len=20):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len-1].flatten())  # ensure (3,), not (3,3)
    return np.array(X_seq), np.array(y_seq)

def load_dataset(folder, seq_len=20):
    X_all, y_all = [], []
    import glob
    for f in glob.glob(f"{folder}/*.npz"):
        data = np.load(f)
        X, y = data['X'], data['y']
        X_seq, y_seq = build_sequences(X, y, seq_len)
        X_all.append(X_seq)
        y_all.append(y_seq)
    X_all = np.concatenate(X_all, axis=0)
    y_all = np.concatenate(y_all, axis=0)
    return X_all, y_all

# 2. Load and prepare sequences
X, y = load_dataset("/content/drive/MyDrive/pid_autotuning/data/train/", seq_len=20)

print("X shape:", X.shape)
print("y shape:", y.shape)
print(model(X[:1]).shape)

# Train the model
# You can adjust the number of epochs and batch size as needed
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)
model.save(f"/content/drive/MyDrive/pid_autotuning/models/{model_name}.h5")

print("Training complete.")

X shape: (4190, 20, 15)
y shape: (4190, 9)
(1, 9)
Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 159.1633 - mae: 9.5691 - val_loss: 0.0974 - val_mae: 0.2166
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0596 - mae: 0.1662 - val_loss: 0.0077 - val_mae: 0.0587
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0115 - mae: 0.0719 - val_loss: 0.0042 - val_mae: 0.0432
Epoch 4/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0062 - mae: 0.0514 - val_loss: 0.0029 - val_mae: 0.0361
Epoch 5/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0044 - mae: 0.0408 - val_loss: 0.0021 - val_mae: 0.0290
Epoch 6/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0035 - mae: 0.0344 - val_loss: 0.0016 - val_mae: 0.0263
Epoch 7/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0026 - mae: 0.0286 - val_loss: 0.0014 - val_mae: 0.0214
Epoch 8/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0020 - mae: 0.0254 - val_loss: 0.0012 - val_mae: 0.0230
Epoc

Training complete.


In [ ]:
X, _ = load_dataset("/content/drive/MyDrive/pid_autotuning/data/to_predict", seq_len=20)
y_pred = model.predict(X, verbose=1)
mean_gains = np.mean(y_pred, axis=0)
print(np.round(mean_gains, 2))

print(f'Kp = np.array([{mean_gains[0]}, {mean_gains[1]}, {mean_gains[2]}])')
print(f'Ki = np.array([{mean_gains[3]}, {mean_gains[4]}, {mean_gains[5]}])')
print(f'Kd = np.array([{mean_gains[6]}, {mean_gains[7]}, {mean_gains[8]}])')

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[21.18 20.46 21.23 18.13 16.01 18.14 -0.05  0.06  0.09]
Kp = np.array([21.184783935546875, 20.463106155395508, 21.234703063964844])
Ki = np.array([18.126968383789062, 16.013917922973633, 18.141902923583984])
Kd = np.array([-0.04748089611530304, 0.05784325301647186, 0.08506572991609573])
